In [4]:
import pandas as pd
import numpy as np
import urllib.request, json
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Загрузить данные из файлов
matches = pd.read_csv('table/table wtih team, kills from 1.09.13 to 22.01.18.csv', index_col=0)
matches = matches.drop(['radiant_id','radiant_name', 'dire_id', 'dire_name','kills'],axis=1).reindex()

In [5]:
heroes = pd.read_csv('All_Heroes.csv')
heroes[:3]

,id,attack_type,id.1,localized_name,primary_attr,roles,Disabler,Nuker,Carry,Initiator,Escape,Durable,Support,Pusher,Jungler
0,1,Melee,1,Anti-Mage,agi,"['Carry', 'Escape', 'Nuker']",0,1,1,0,1,0,0,0,0
1,2,Melee,2,Axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']",1,0,0,1,0,1,0,0,1
2,3,Ranged,3,Bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']",1,1,0,0,0,1,1,0,0


In [6]:
heroes['Disabler'][heroes['id'] == 4].item()

1

In [7]:
columns = ['attac_type1','Disabler1','Nuker1','Carry1','Initiator1','Escape1','Durable1','Support1','Pusher1','Jungler1',
           'attac_type2','Disabler2','Nuker2','Carry2','Initiator2','Escape2','Durable2','Support2','Pusher2','Jungler2',
           'attac_type3','Disabler3','Nuker3','Carry3','Initiator3','Escape3','Durable3','Support3','Pusher3','Jungler3',
           'attac_type4','Disabler4','Nuker4','Carry4','Initiator4','Escape4','Durable4','Support4','Pusher4','Jungler4',
           'attac_type5','Disabler5','Nuker5','Carry5','Initiator5','Escape5','Durable5','Support5','Pusher5','Jungler5',
           'attac_type6','Disabler6','Nuker6','Carry6','Initiator6','Escape6','Durable6','Support6','Pusher6','Jungler6',
           'attac_type7','Disabler7','Nuker7','Carry7','Initiator7','Escape7','Durable7','Support7','Pusher7','Jungler7',
           'attac_type8','Disabler8','Nuker8','Carry8','Initiator8','Escape8','Durable8','Support8','Pusher8','Jungler8',
           'attac_type9','Disabler9','Nuker9','Carry9','Initiator9','Escape9','Durable9','Support9','Pusher9','Jungler9',
     'attac_type10','Disabler10','Nuker10','Carry10','Initiator10','Escape10','Durable10','Support10','Pusher10','Jungler10']

list_scripts = ['Disabler', 'Nuker', 'Carry', 'Initiator', 'Escape', 'Durable',
                'Support', 'Pusher', 'Jungler']

In [ ]:
# функция для перевода в бинарное состояние типа атаки
def atac(type_at):
    if type_at == 'Melee':
        return 0
    else:
        return 1
# df в который записыватеся пик команда с росписью кажого героя    
df_script_her = pd.DataFrame(columns=columns)
# df из которого сотавляется новый df
little = matches

for index in range(len(little)):
    # для radiant
    for i in range(1,6):
        id_her = little['radiant' + str(i)].loc[index]
        df_script_her.loc[index, ('attac_type' + str(i))] = atac(heroes['attack_type'][heroes['id'] == id_her].item())
        for col in list_scripts:
            sc = heroes[col][(heroes['id'] == id_her)].item()
            df_script_her.loc[index, (col + str(i))] = sc
    # для dire
    for i in range(1,6):
        id_her = little['dire' + str(i)].loc[index]
        df_script_her.loc[index, ('attac_type' + str(i+5))] = atac(heroes['attack_type'][heroes['id'] == id_her].item())
        for col in list_scripts:
            sc = heroes[col][(heroes['id'] == id_her)].item()
            df_script_her.loc[index, (col + str(i+5))] = sc      


In [ ]:
df_script_her['radiant_win'] = little['radiant_win']
# df_script_her['radiant_id'] = little['radiant_id']
# df_script_her['dire_id'] = little['dire_id']
len(df_script_her)
#df_script_her.to_csv('table/table wtih  carry or support  from 1.09.13 to 22.01.18.csv')

In [71]:
df_script = pd.read_csv('table/table with carry or support  from 1.09.13 to 22.01.18.csv', index_col=0)


In [72]:
# Добавить суммы все names по каждой команду
names = ['Disabler','Nuker','Carry','Initiator','Escape','Durable','Support','Pusher','Jungler']

for name in names:
    df_script[name + '_radiant'] = (df_script[name + '1'] + df_script[name + '2'] + df_script[name + '3'] + 
                                   df_script[name + '4'] + df_script[name + '5'])
    df_script[name + '_dire'] = (df_script[name + '6'] + df_script[name + '7'] + df_script[name + '8'] + 
                                   df_script[name + '9'] + df_script[name + '10'])


In [77]:
df = df_script.loc[:,'radiant_win':].fillna(0)

df.to_csv('table/table with SUMM for two team carry or support  from 1.09.13 to 22.01.18.csv')